### Problem statement and data overview

The data for this project has been taken from Kaggle competition **[Home credit default risk](https://www.kaggle.com/c/home-credit-default-risk)**. The objective of this competition was to use data from various sources like credit bureau, previous loan applications, previous loan performance, and current application data in order to predict the probability of default of the applicant. The output of this model (in form of scores or risk bands) can then be used to make a credit decision (approve/reject) regarding the application. 

In this notebook we focus on importing all the datasets and understanding the data. We will create features from each of these datasets and map these onto our training data which will be evantually used to fit the model.

In [67]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
pd.options.display.max_columns = None

import warnings
warnings.filterwarnings('ignore')

### Importing train data

**application_{train|test}.csv :** This is the main table, broken into two files for Train (with TARGET) and Test (without TARGET). Static data for all applications. One row represents one loan in our data sample. "*SK_ID_CURR*" is the primary key representing each application. We will have to rollup the data on the other datasets on this column in order to map it onto the training data.

In [16]:
application_train = pd.read_csv("application_train.csv")

In [17]:
application_train.shape

(307511, 122)

### Balance data for pos & cash loan

**POS_CASH_balance.csv :** Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit. This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

In [3]:
pos_cash_bal = pd.read_csv("POS_CASH_balance.csv")

In [7]:
pos_cash_bal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001358 entries, 0 to 10001357
Data columns (total 8 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   SK_ID_PREV             int64  
 1   SK_ID_CURR             int64  
 2   MONTHS_BALANCE         int64  
 3   CNT_INSTALMENT         float64
 4   CNT_INSTALMENT_FUTURE  float64
 5   NAME_CONTRACT_STATUS   object 
 6   SK_DPD                 int64  
 7   SK_DPD_DEF             int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 610.4+ MB


In [8]:
pos_cash_bal.shape

(10001358, 8)

In [11]:
print('Unique prev ids in the pos cash bal: ', len(pos_cash_bal.SK_ID_PREV.unique()))
print('Unique curr ids in the pos cash bal: ', len(pos_cash_bal.SK_ID_CURR.unique()))

Unique prev ids in the pos cash bal:  936325
Unique curr ids in the pos cash bal:  337252


In [19]:
'Curr IDs from credit pos cash balance present in train set: {0}'. \
format(len(pos_cash_bal[pos_cash_bal.SK_ID_CURR.isin(application_train.SK_ID_CURR)]['SK_ID_CURR'].unique()))

'Curr IDs from credit pos cash balance present in train set: 289444'

### POS cash loans aggregated variables

Since this data gives monthly balance snapshots of the applicant on previous pos or cash loan with Home credit we can use it to generate aggregated features at **SK_ID_CURR** level. Some examples are Active loans 6/12/24/26 months prior to current loan,  Completed loans 6/12/24/26 months prior, delinquent loans 6/12/24/26 months prior. The function below creates these variables and rolls up the data on column **SK_ID_CURR**

In [28]:
def pos_cash_lagged_features(pos_cash_bal):
    
    pos_cash_grpby = \
    pos_cash_bal.assign(
    active_loans_curr = np.where((pos_cash_bal.MONTHS_BALANCE == -1) & (pos_cash_bal.NAME_CONTRACT_STATUS == "Active"),1,0),
    active_loans_6m_prior = np.where((pos_cash_bal.MONTHS_BALANCE == -6) & (pos_cash_bal.NAME_CONTRACT_STATUS == "Active"),1,0),
    active_loans_12m_prior = np.where((pos_cash_bal.MONTHS_BALANCE == -12) & (pos_cash_bal.NAME_CONTRACT_STATUS == "Active"),1,0),
    active_loans_24m_prior = np.where((pos_cash_bal.MONTHS_BALANCE == -24) & (pos_cash_bal.NAME_CONTRACT_STATUS == "Active"),1,0),
    active_loans_36m_prior = np.where((pos_cash_bal.MONTHS_BALANCE == -36) & (pos_cash_bal.NAME_CONTRACT_STATUS == "Active"),1,0),       
    active_loans_48m_prior = np.where((pos_cash_bal.MONTHS_BALANCE == -48) & (pos_cash_bal.NAME_CONTRACT_STATUS == "Active"),1,0),

    completed_loans_curr = np.where((pos_cash_bal.MONTHS_BALANCE == -1) & (pos_cash_bal.NAME_CONTRACT_STATUS == "Completed"),1,0),
    completed_loans_6m_prior = np.where((pos_cash_bal.MONTHS_BALANCE == -6) & (pos_cash_bal.NAME_CONTRACT_STATUS == "Completed"),1,0),
    completed_loans_12m_prior = np.where((pos_cash_bal.MONTHS_BALANCE == -12) & (pos_cash_bal.NAME_CONTRACT_STATUS == "Completed"),1,0),
    completed_loans_24m_prior = np.where((pos_cash_bal.MONTHS_BALANCE == -24) & (pos_cash_bal.NAME_CONTRACT_STATUS == "Completed"),1,0),
    completed_loans_36m_prior = np.where((pos_cash_bal.MONTHS_BALANCE == -36) & (pos_cash_bal.NAME_CONTRACT_STATUS == "Completed"),1,0),       
    completed_loans_48m_prior = np.where((pos_cash_bal.MONTHS_BALANCE == -48) & (pos_cash_bal.NAME_CONTRACT_STATUS == "Completed"),1,0),

    delq_trades_curr = np.where((pos_cash_bal.SK_DPD_DEF > 0) & (pos_cash_bal.MONTHS_BALANCE == -1), 1, 0),
    delq_trades_6m_prior  = np.where((pos_cash_bal.SK_DPD_DEF > 0) & (pos_cash_bal.MONTHS_BALANCE == -6), 1, 0),
    delq_trades_12m_prior = np.where((pos_cash_bal.SK_DPD_DEF > 0) & (pos_cash_bal.MONTHS_BALANCE == -12), 1, 0),
    delq_trades_24m_prior = np.where((pos_cash_bal.SK_DPD_DEF > 0) & (pos_cash_bal.MONTHS_BALANCE == -24), 1, 0),
    delq_trades_36m_prior = np.where((pos_cash_bal.SK_DPD_DEF > 0) & (pos_cash_bal.MONTHS_BALANCE == -36), 1, 0),
    delq_trades_48m_prior = np.where((pos_cash_bal.SK_DPD_DEF > 0) & (pos_cash_bal.MONTHS_BALANCE == -48), 1, 0)
    )

    pos_cash_grpby = \
    pos_cash_grpby \
    .groupby('SK_ID_CURR').agg({
       "SK_ID_PREV" : "nunique",
    
       'active_loans_curr':"sum",
       'active_loans_6m_prior' : "sum",
       'active_loans_12m_prior' : "sum",
       'active_loans_24m_prior' : "sum",
       'active_loans_36m_prior' : "sum",
       'active_loans_48m_prior' : "sum",
      
       "completed_loans_curr": "sum", 
        'completed_loans_6m_prior' : "sum",
        'completed_loans_12m_prior' : "sum",
        'completed_loans_24m_prior' : "sum",
        'completed_loans_36m_prior' : "sum",
        'completed_loans_48m_prior' : "sum",
      
       'delq_trades_curr': "sum",
        'delq_trades_6m_prior' : "sum",
        'delq_trades_12m_prior' : "sum",
        'delq_trades_24m_prior' : "sum",
        'delq_trades_36m_prior' : "sum",
        'delq_trades_48m_prior' : "sum",
     }).reset_index()

    pos_cash_grpby.rename(columns = {'SK_ID_PREV': 'total_unique_trades'},inplace=True)
    
    #pending installment variable
    completed_ids = pos_cash_bal[pos_cash_bal.NAME_CONTRACT_STATUS == 'Completed']['SK_ID_PREV'].unique()
    installment_var = pos_cash_bal[~pos_cash_bal.SK_ID_PREV.isin(completed_ids)]
    installment_var.sort_values(['SK_ID_PREV','MONTHS_BALANCE'],ascending = [True,False],inplace=True)
    installment_var.drop_duplicates('SK_ID_PREV', keep='first', inplace = True)
    installment_var = installment_var.groupby('SK_ID_CURR').agg({'CNT_INSTALMENT':sum,'CNT_INSTALMENT_FUTURE':sum}).reset_index()
    installment_var['prop_pending_installment'] = np.round(installment_var.CNT_INSTALMENT_FUTURE/installment_var.CNT_INSTALMENT,2)

    return pos_cash_grpby, installment_var


In [69]:
# Executing the above funtion to create agg variables
pos_cash_grpby, installment_var = pos_cash_lagged_features(pos_cash_bal)

In [32]:
print(pos_cash_grpby.shape)
print(installment_var.shape)

(337252, 20)
(181772, 4)


In [26]:
pos_cash_grpby.to_csv('pos_cash_grpby.csv', index = False)
installment_var.to_csv('installment_var.csv', index = False)

In [6]:
import gc
gc.collect()

1497

### Balance data for credit cards

**credit_card_balance.csv :** Monthly balance snapshots of previous credit cards that the applicant has with Home Credit. This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

In [21]:
credit_card_balance = pd.read_csv('credit_card_balance.csv')

In [22]:
credit_card_balance.shape

(3840312, 23)

In [23]:
credit_card_balance.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840312 entries, 0 to 3840311
Data columns (total 23 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   SK_ID_PREV                  3840312 non-null  int64  
 1   SK_ID_CURR                  3840312 non-null  int64  
 2   MONTHS_BALANCE              3840312 non-null  int64  
 3   AMT_BALANCE                 3840312 non-null  float64
 4   AMT_CREDIT_LIMIT_ACTUAL     3840312 non-null  int64  
 5   AMT_DRAWINGS_ATM_CURRENT    3090496 non-null  float64
 6   AMT_DRAWINGS_CURRENT        3840312 non-null  float64
 7   AMT_DRAWINGS_OTHER_CURRENT  3090496 non-null  float64
 8   AMT_DRAWINGS_POS_CURRENT    3090496 non-null  float64
 9   AMT_INST_MIN_REGULARITY     3535076 non-null  float64
 10  AMT_PAYMENT_CURRENT         3072324 non-null  float64
 11  AMT_PAYMENT_TOTAL_CURRENT   3840312 non-null  float64
 12  AMT_RECEIVABLE_PRINCIPAL    3840312 non-null  float64
 1

In [24]:
print('Unique prev ids in the pos cash bal: ', len(credit_card_balance.SK_ID_PREV.unique()))
print('Unique curr ids in the pos cash bal: ', len(credit_card_balance.SK_ID_CURR.unique()))

Unique prev ids in the pos cash bal:  104307
Unique curr ids in the pos cash bal:  103558


In [25]:
'IDs from credit card bal present in train set: {0}'. \
format(len(credit_card_balance[credit_card_balance.SK_ID_CURR.isin(application_train.SK_ID_CURR)]['SK_ID_CURR'].unique()))

'IDs from credit card bal present in train set: 86905'

### CC aggregated variables

Since this data gives monthly of the applicant behaviur on credit card with Home credit. This gives us month on month information on outstanding balances, dpd, utilization, drawing etc. of the customer. Some features are active cards, closed cards, min/max,avg utlization, times min amt due is unpaid.

In [29]:
def credit_card_lagged_features(credit_card_balance):
    
    cc_grpby = \
    credit_card_balance.assign(
    active_cc_trades_curr = np.where((credit_card_balance.MONTHS_BALANCE == -1) & (credit_card_balance.NAME_CONTRACT_STATUS == "Active"),1,0),
    active_cc_trades_6m_prior = np.where((credit_card_balance.MONTHS_BALANCE == -6) & (credit_card_balance.NAME_CONTRACT_STATUS == "Active"),1,0),
    active_cc_trades_12m_prior = np.where((credit_card_balance.MONTHS_BALANCE == -12) & (credit_card_balance.NAME_CONTRACT_STATUS == "Active"),1,0),
    active_cc_trades_24m_prior = np.where((credit_card_balance.MONTHS_BALANCE == -24) & (credit_card_balance.NAME_CONTRACT_STATUS == "Active"),1,0),
    active_cc_trades_36m_prior = np.where((credit_card_balance.MONTHS_BALANCE == -36) & (credit_card_balance.NAME_CONTRACT_STATUS == "Active"),1,0),       
    active_cc_trades_48m_prior = np.where((credit_card_balance.MONTHS_BALANCE == -48) & (credit_card_balance.NAME_CONTRACT_STATUS == "Active"),1,0),

    completed_cc_trades_curr = np.where((credit_card_balance.MONTHS_BALANCE == -1) & (credit_card_balance.NAME_CONTRACT_STATUS == "Completed"),1,0),
    completed_cc_trades_6m_prior = np.where((credit_card_balance.MONTHS_BALANCE == -6) & (credit_card_balance.NAME_CONTRACT_STATUS == "Completed"),1,0),
    completed_cc_trades_12m_prior = np.where((credit_card_balance.MONTHS_BALANCE == -12) & (credit_card_balance.NAME_CONTRACT_STATUS == "Completed"),1,0),
    completed_cc_trades_24m_prior = np.where((credit_card_balance.MONTHS_BALANCE == -24) & (credit_card_balance.NAME_CONTRACT_STATUS == "Completed"),1,0),
    completed_cc_trades_36m_prior = np.where((credit_card_balance.MONTHS_BALANCE == -36) & (credit_card_balance.NAME_CONTRACT_STATUS == "Completed"),1,0),       
    completed_cc_trades_48m_prior = np.where((credit_card_balance.MONTHS_BALANCE == -48) & (credit_card_balance.NAME_CONTRACT_STATUS == "Completed"),1,0),

    delq_cc_loans_curr = np.where((credit_card_balance.SK_DPD_DEF > 0) & (credit_card_balance.MONTHS_BALANCE == -1), 1, 0),
    delq_cc_loans_6m_prior  = np.where((credit_card_balance.SK_DPD_DEF > 0) & (credit_card_balance.MONTHS_BALANCE == -6), 1, 0),
    delq_cc_loans_12m_prior = np.where((credit_card_balance.SK_DPD_DEF > 0) & (credit_card_balance.MONTHS_BALANCE == -12), 1, 0),
    delq_cc_loans_24m_prior = np.where((credit_card_balance.SK_DPD_DEF > 0) & (credit_card_balance.MONTHS_BALANCE == -24), 1, 0),
    delq_cc_loans_36m_prior = np.where((credit_card_balance.SK_DPD_DEF > 0) & (credit_card_balance.MONTHS_BALANCE == -36), 1, 0),
    delq_cc_loans_48m_prior = np.where((credit_card_balance.SK_DPD_DEF > 0) & (credit_card_balance.MONTHS_BALANCE == -48), 1, 0),

    min_due_unpaid_prior_6m = np.where((credit_card_balance.AMT_INST_MIN_REGULARITY > credit_card_balance.AMT_PAYMENT_CURRENT) & (credit_card_balance.MONTHS_BALANCE >= -6),1,0),
    min_due_unpaid_prior_12m = np.where((credit_card_balance.AMT_INST_MIN_REGULARITY > credit_card_balance.AMT_PAYMENT_CURRENT) & (credit_card_balance.MONTHS_BALANCE >= -12),1,0),
    min_due_unpaid_prior_24m = np.where((credit_card_balance.AMT_INST_MIN_REGULARITY > credit_card_balance.AMT_PAYMENT_CURRENT) & (credit_card_balance.MONTHS_BALANCE >= -24),1,0),
    min_due_unpaid_prior_36m = np.where((credit_card_balance.AMT_INST_MIN_REGULARITY > credit_card_balance.AMT_PAYMENT_CURRENT) & (credit_card_balance.MONTHS_BALANCE >= -36),1,0),
    min_due_unpaid_prior_48m = np.where((credit_card_balance.AMT_INST_MIN_REGULARITY > credit_card_balance.AMT_PAYMENT_CURRENT) & (credit_card_balance.MONTHS_BALANCE >= -48),1,0),

    util_prior_6m = np.where((credit_card_balance.MONTHS_BALANCE >= -6),   np.round(credit_card_balance.AMT_BALANCE/credit_card_balance.AMT_CREDIT_LIMIT_ACTUAL * 100, 1) ,np.nan),
    util_prior_12m = np.where((credit_card_balance.MONTHS_BALANCE >= -12), np.round(credit_card_balance.AMT_BALANCE/credit_card_balance.AMT_CREDIT_LIMIT_ACTUAL * 100, 1) ,np.nan),    
    util_prior_24m = np.where((credit_card_balance.MONTHS_BALANCE >= -24), np.round(credit_card_balance.AMT_BALANCE/credit_card_balance.AMT_CREDIT_LIMIT_ACTUAL * 100, 1) ,np.nan),   
    util_prior_36m = np.where((credit_card_balance.MONTHS_BALANCE >= -36), np.round(credit_card_balance.AMT_BALANCE/credit_card_balance.AMT_CREDIT_LIMIT_ACTUAL * 100, 1) ,np.nan),   
    util_prior_48m = np.where((credit_card_balance.MONTHS_BALANCE >= -48), np.round(credit_card_balance.AMT_BALANCE/credit_card_balance.AMT_CREDIT_LIMIT_ACTUAL * 100, 1) ,np.nan)     
    )

    cc_grpby = \
    cc_grpby \
    .groupby('SK_ID_CURR').agg({
       "SK_ID_PREV" : "nunique",
    
       'active_cc_trades_curr':"sum",
       'active_cc_trades_6m_prior' : "sum",
       'active_cc_trades_12m_prior' : "sum",
       'active_cc_trades_24m_prior' : "sum",
       'active_cc_trades_36m_prior' : "sum",
       'active_cc_trades_48m_prior' : "sum",
      
        "completed_cc_trades_curr": "sum", 
        'completed_cc_trades_6m_prior' : "sum",
        'completed_cc_trades_12m_prior' : "sum",
        'completed_cc_trades_24m_prior' : "sum",
        'completed_cc_trades_36m_prior' : "sum",
        'completed_cc_trades_48m_prior' : "sum",
      
        'delq_cc_loans_curr': "sum",
        'delq_cc_loans_6m_prior' : "sum",
        'delq_cc_loans_12m_prior' : "sum",
        'delq_cc_loans_24m_prior' : "sum",
        'delq_cc_loans_36m_prior' : "sum",
        'delq_cc_loans_48m_prior' : "sum",
    
        'min_due_unpaid_prior_6m' : 'sum',
        'min_due_unpaid_prior_12m' : 'sum',
        'min_due_unpaid_prior_24m' : 'sum',
        'min_due_unpaid_prior_36m' : 'sum',
        'min_due_unpaid_prior_48m' : 'sum',
    
        'util_prior_6m'  : ['mean','max','min'],
        'util_prior_12m' : ['mean','max','min'],
        'util_prior_24m' : ['mean','max','min'],
        'util_prior_36m' : ['mean','max','min'],
        'util_prior_48m' : ['mean','max','min']
    
     })

    cc_grpby.rename(columns = {'SK_ID_PREV': 'total_unique_cc_trades'},inplace=True)
    
    cols = ['_'.join(x) for x in cc_grpby.columns.ravel()]
    cols1 = [col.replace("_sum", "") for col in cols]
    cols2 = [col.replace("_nunique", "") for col in cols1]
    
    cc_grpby.columns = cols2
    cc_grpby = cc_grpby.reset_index()
    
    return cc_grpby


In [30]:
cc_grpby = credit_card_lagged_features(credit_card_balance)

In [31]:
cc_grpby.shape

(103558, 40)

In [ ]:
cc_grpby.to_csv('cc_grpby.csv', index = False)

### Previous application data
**previous_application.csv :** All previous applications for Home Credit loans of clients who have loans in our sample. There is one row for each previous application related to loans in our data sample. 

In [33]:
previous_application = pd.read_csv('previous_application.csv')

In [34]:
previous_application.shape

(1670214, 37)

In [39]:
print('Unique curr ids in the previous application data: ', len(previous_application.SK_ID_CURR.unique()))

Unique curr ids in the previous application data:  338857


In [59]:
'Curr ids from previous applcation data present in train data: {}'\
.format(application_train.SK_ID_CURR.isin(previous_application.SK_ID_CURR.unique()).sum())

'Curr ids from previous applcation data present in train data: 291057'

In [64]:
previous_application.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 37 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1670214 non-null  int64  
 1   SK_ID_CURR                   1670214 non-null  int64  
 2   NAME_CONTRACT_TYPE           1670214 non-null  object 
 3   AMT_ANNUITY                  1297979 non-null  float64
 4   AMT_APPLICATION              1670214 non-null  float64
 5   AMT_CREDIT                   1670213 non-null  float64
 6   AMT_DOWN_PAYMENT             774370 non-null   float64
 7   AMT_GOODS_PRICE              1284699 non-null  float64
 8   WEEKDAY_APPR_PROCESS_START   1670214 non-null  object 
 9   HOUR_APPR_PROCESS_START      1670214 non-null  int64  
 10  FLAG_LAST_APPL_PER_CONTRACT  1670214 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY       1670214 non-null  int64  
 12  RATE_DOWN_PAYMENT            774370 non-nu

### Previous application grouped variables
Here we can create aggregate features like count of previous applied/approved/rejected loans, avg ticket size/LTV/EMI/Tenor of loans, count of type of loan availed, avg time gap between two approved loans

In [43]:
def prev_app_lagged_features(previous_application):
    
    previous_application_grpby = \
    previous_application.assign \
    (
    prev_app_approved_count = np.where(previous_application.NAME_CONTRACT_STATUS == 'Approved',1,0),
    prev_app_canceled_count = np.where(previous_application.NAME_CONTRACT_STATUS == 'Canceled',1,0),
    prev_app_Refused_count  = np.where(previous_application.NAME_CONTRACT_STATUS.isin(['Refused','Unused offer']),1,0),

    prev_app_avg_ticket_size   = np.where(previous_application.NAME_CONTRACT_STATUS == 'Approved', previous_application.AMT_CREDIT, np.NaN),
    prev_app_avg_LTV           = np.where(previous_application.NAME_CONTRACT_STATUS == 'Approved', np.round(previous_application.AMT_APPLICATION/previous_application.AMT_GOODS_PRICE * 100,2), np.NaN),
    prev_app_avg_EMI           = np.where(previous_application.NAME_CONTRACT_STATUS == 'Approved', previous_application.AMT_ANNUITY, np.NaN),
    prev_app_avg_down_pmt_rate = np.where(previous_application.NAME_CONTRACT_STATUS == 'Approved', np.round(previous_application.RATE_DOWN_PAYMENT,2), np.NaN),
    prev_app_avg_tenor         = np.where(previous_application.NAME_CONTRACT_STATUS == 'Approved', previous_application.CNT_PAYMENT, np.NaN),

    prev_app_cash_loans_approved      = np.where((previous_application.NAME_CONTRACT_TYPE == 'Cash loans') & (previous_application.NAME_CONTRACT_STATUS == 'Approved'), 1, 0),
    prev_app_consumer_loans_approved  = np.where((previous_application.NAME_CONTRACT_TYPE == 'Consumer loans') & (previous_application.NAME_CONTRACT_STATUS == 'Approved'), 1, 0),
    prev_app_revolving_loans_approved = np.where((previous_application.NAME_CONTRACT_TYPE == 'Revolving loans') & (previous_application.NAME_CONTRACT_STATUS == 'Approved'), 1, 0),

    prev_app_mode_portfolio    = np.where((previous_application.NAME_CONTRACT_STATUS == 'Approved'), previous_application.NAME_PORTFOLIO,np.NaN),    
    prev_app_mode_channel_type = np.where((previous_application.NAME_CONTRACT_STATUS == 'Approved'), previous_application.CHANNEL_TYPE,np.NaN),
    prev_app_mode_yield_grp    = np.where((previous_application.NAME_CONTRACT_STATUS == 'Approved'), previous_application.NAME_YIELD_GROUP,np.NaN),
    prev_app_mode_seller_ind   = np.where((previous_application.NAME_CONTRACT_STATUS == 'Approved'), previous_application.NAME_SELLER_INDUSTRY,np.NaN)    

    )
    
    previous_application_grpby = \
    previous_application_grpby.groupby('SK_ID_CURR').\
    agg({
    'prev_app_approved_count': sum,
    'prev_app_canceled_count': sum,
    'prev_app_Refused_count' : sum,

    'prev_app_avg_ticket_size'  : 'mean',
    'prev_app_avg_LTV'          : 'mean',
    'prev_app_avg_EMI'          : 'mean',
    'prev_app_avg_down_pmt_rate': 'mean',
    'prev_app_avg_tenor'        : 'mean',

    'prev_app_cash_loans_approved'      : sum,
    'prev_app_consumer_loans_approved'  : sum,
    'prev_app_revolving_loans_approved' : sum,

    'prev_app_mode_portfolio'    : lambda x: x.mode(),
    'prev_app_mode_channel_type' : lambda x: x.mode(),
    'prev_app_mode_yield_grp'    : lambda x: x.mode(),
    'prev_app_mode_seller_ind'   : lambda x: x.mode()
    })
    
    previous_application_grpby = previous_application_grpby.reset_index()
    
    trade_gap = previous_application[previous_application.NAME_CONTRACT_STATUS == 'Approved']
    trade_gap = trade_gap[['SK_ID_CURR','SK_ID_PREV','NAME_CONTRACT_STATUS','DAYS_DECISION']].sort_values(['SK_ID_CURR','DAYS_DECISION'])
    trade_gap['gap'] = trade_gap.groupby('SK_ID_CURR')['DAYS_DECISION'].diff()
    
    avg_trade_gap = trade_gap.groupby('SK_ID_CURR').agg(avg_gap = ('gap','mean')).reset_index()
    avg_trade_gap['avg_gap'] = np.abs(avg_trade_gap.avg_gap)

    return previous_application_grpby, avg_trade_gap
    

In [44]:
previous_application_grpby, avg_trade_gap = prev_app_lagged_features(previous_application)

In [46]:
print(previous_application_grpby.shape)
print(avg_trade_gap.shape)

(338857, 16)
(337698, 2)


In [48]:
previous_application_grpby.to_csv('previous_application_grpby.csv', index = False)
avg_trade_gap.to_csv('avg_trade_gap.csv',index = False)

### Installment payment info

**installments_payments.csv :** Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample. There is a) one row for every payment that was made plus b) one row each for missed payment. One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.

In [47]:
instl_pmt = pd.read_csv("installments_payments.csv")

In [48]:
instl_pmt.shape

(13605401, 8)

In [49]:
print('Unique prev ids in the installment data:', len(instl_pmt.SK_ID_PREV.unique()))
print('Unique curr ids in the installment data:', len(instl_pmt.SK_ID_CURR.unique()))

Unique prev ids in the installment data: 997752
Unique curr ids in the installment data: 339587


In [61]:
'Curr ids from installment data present in train data: {}'\
.format(instl_pmt.drop_duplicates('SK_ID_CURR')['SK_ID_CURR'].isin(application_train.SK_ID_CURR).sum())

'Curr ids from installment data present in train data: 291643'

In [233]:
instl_pmt.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
 #   Column                  Non-Null Count     Dtype  
---  ------                  --------------     -----  
 0   SK_ID_PREV              13605401 non-null  int64  
 1   SK_ID_CURR              13605401 non-null  int64  
 2   NUM_INSTALMENT_VERSION  13605401 non-null  float64
 3   NUM_INSTALMENT_NUMBER   13605401 non-null  int64  
 4   DAYS_INSTALMENT         13605401 non-null  float64
 5   DAYS_ENTRY_PAYMENT      13602496 non-null  float64
 6   AMT_INSTALMENT          13605401 non-null  float64
 7   AMT_PAYMENT             13602496 non-null  float64
dtypes: float64(5), int64(3)
memory usage: 830.4 MB


### Installment aggregate variables

This data is used to create aggregate variables showing payment behaviour of the customer. Some examples of features are count of late payments, partial payments, pre-payment over the past 1/12/24/36 months.

In [64]:
def installment_lagged_features(instl_pmt):
    
    instl_pmt_grp1 = instl_pmt.groupby(['SK_ID_PREV','SK_ID_CURR','NUM_INSTALMENT_VERSION','NUM_INSTALMENT_NUMBER']) \
                              .agg({'AMT_INSTALMENT':max,'AMT_PAYMENT': sum, 'DAYS_INSTALMENT':max,'DAYS_ENTRY_PAYMENT':max}) \
                              .reset_index()

    instl_pmt_grp2 = \
    instl_pmt_grp1.assign(
    times_late_pmt_past_6m  = np.where((instl_pmt_grp1.DAYS_INSTALMENT  >= -180) & (instl_pmt_grp1.DAYS_INSTALMENT < instl_pmt_grp1.DAYS_ENTRY_PAYMENT),1,0),
    times_late_pmt_past_12m = np.where((instl_pmt_grp1.DAYS_INSTALMENT >= -365) &  (instl_pmt_grp1.DAYS_INSTALMENT < instl_pmt_grp1.DAYS_ENTRY_PAYMENT),1,0),
    times_late_pmt_past_24m = np.where((instl_pmt_grp1.DAYS_INSTALMENT >= -730) &  (instl_pmt_grp1.DAYS_INSTALMENT < instl_pmt_grp1.DAYS_ENTRY_PAYMENT),1,0),   
    times_late_pmt_past_36m = np.where((instl_pmt_grp1.DAYS_INSTALMENT >= -1095) & (instl_pmt_grp1.DAYS_INSTALMENT < instl_pmt_grp1.DAYS_ENTRY_PAYMENT),1,0),
    times_late_pmt_past_48m = np.where((instl_pmt_grp1.DAYS_INSTALMENT >= -1460) & (instl_pmt_grp1.DAYS_INSTALMENT < instl_pmt_grp1.DAYS_ENTRY_PAYMENT),1,0),

    times_partial_pmt_6m = np.where((instl_pmt_grp1.DAYS_INSTALMENT  >= -180) &  (round(instl_pmt_grp1.AMT_PAYMENT, 2)  < round(instl_pmt_grp1.AMT_INSTALMENT,2)),1,0),
    times_partial_pmt_12m = np.where((instl_pmt_grp1.DAYS_INSTALMENT >= -365) &  (round(instl_pmt_grp1.AMT_PAYMENT, 2)  < round(instl_pmt_grp1.AMT_INSTALMENT,2)),1,0),    
    times_partial_pmt_24m = np.where((instl_pmt_grp1.DAYS_INSTALMENT >= -730) &  (round(instl_pmt_grp1.AMT_PAYMENT, 2)  < round(instl_pmt_grp1.AMT_INSTALMENT,2)),1,0),    
    times_partial_pmt_36m = np.where((instl_pmt_grp1.DAYS_INSTALMENT >= -1095) & (round(instl_pmt_grp1.AMT_PAYMENT, 2)  < round(instl_pmt_grp1.AMT_INSTALMENT,2)),1,0),    
    times_partial_pmt_48m = np.where((instl_pmt_grp1.DAYS_INSTALMENT >= -1460) & (round(instl_pmt_grp1.AMT_PAYMENT, 2)  < round(instl_pmt_grp1.AMT_INSTALMENT,2)),1,0),

    times_pre_pmt_6m = np.where((instl_pmt_grp1.DAYS_INSTALMENT  >= -180) & (round(instl_pmt_grp1.AMT_PAYMENT, 2)  > round(instl_pmt_grp1.AMT_INSTALMENT,2)),1,0),
    times_pre_pmt_12m = np.where((instl_pmt_grp1.DAYS_INSTALMENT >= -365) & (round(instl_pmt_grp1.AMT_PAYMENT, 2)  > round(instl_pmt_grp1.AMT_INSTALMENT,2)),1,0),    
    times_pre_pmt_24m = np.where((instl_pmt_grp1.DAYS_INSTALMENT >= -730) & (round(instl_pmt_grp1.AMT_PAYMENT, 2)  > round(instl_pmt_grp1.AMT_INSTALMENT,2)),1,0),    
    times_pre_pmt_36m = np.where((instl_pmt_grp1.DAYS_INSTALMENT >= -1095) & (round(instl_pmt_grp1.AMT_PAYMENT, 2) > round(instl_pmt_grp1.AMT_INSTALMENT,2)),1,0),    
    times_pre_pmt_48m = np.where((instl_pmt_grp1.DAYS_INSTALMENT >= -1460) & (round(instl_pmt_grp1.AMT_PAYMENT, 2) > round(instl_pmt_grp1.AMT_INSTALMENT,2)),1,0)
    )
    
    instl_pmt_grp2 = \
    instl_pmt_grp2.groupby(['SK_ID_CURR']). \
    agg({
    'times_late_pmt_past_6m'  : sum, 
    'times_late_pmt_past_12m' : sum,
    'times_late_pmt_past_24m' : sum,
    'times_late_pmt_past_36m' : sum,
    'times_late_pmt_past_48m' : sum,

    'times_partial_pmt_6m'  : sum,
    'times_partial_pmt_12m' : sum,
    'times_partial_pmt_24m' : sum,
    'times_partial_pmt_36m' : sum,
    'times_partial_pmt_48m' : sum,

    'times_pre_pmt_6m' : sum,
    'times_pre_pmt_12m' : sum,
    'times_pre_pmt_24m' : sum,
    'times_pre_pmt_36m' : sum,
    'times_pre_pmt_48m' : sum

    }).reset_index()
    
    return instl_pmt_grp2
    

In [65]:
instl_pmt_grp2 = installment_lagged_features(instl_pmt)

In [66]:
instl_pmt_grp2.shape

(339587, 16)

In [ ]:
instl_pmt_grp2.to_csv('instl_pmt_grp2.csv', index = False)

Finally we map data from the above aggrgated datasets on the training data

In [70]:
# Mapping the data on train

application_train_mapped = pd.merge(application_train, pos_cash_grpby, 'left', on = 'SK_ID_CURR', suffixes=('',''))
application_train_mapped = pd.merge(application_train_mapped, installment_var, 'left', on = 'SK_ID_CURR', suffixes=('',''))
application_train_mapped = pd.merge(application_train_mapped, cc_grpby, 'left', on = 'SK_ID_CURR', suffixes=('',''))
application_train_mapped = pd.merge(application_train_mapped, previous_application_grpby, 'left', on = 'SK_ID_CURR', suffixes=('',''))
application_train_mapped = pd.merge(application_train_mapped, avg_trade_gap, 'left', on = 'SK_ID_CURR', suffixes=('',''))
application_train_mapped = pd.merge(application_train_mapped, instl_pmt_grp2, 'left', on = 'SK_ID_CURR', suffixes=('',''))

In [ ]:
# Store for later use
application_train_mapped.to_csv('application_train_mapped.csv')